In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False


In [2]:
if IN_COLAB:
    !rm -rf Projekt_AIM_1
    !git clone https://github.com/Gaurang8200/Projekt_AIM_1.git

Cloning into 'Projekt_AIM_1'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 155 (delta 44), reused 23 (delta 23), pack-reused 100 (from 1)
Receiving objects: 100% (155/155), 94.82 KiB | 1.90 MiB/s, done.
Resolving deltas: 100% (114/114), done.


In [3]:
if IN_COLAB:
    !pip3 install -r Projekt_AIM_1/requirements.txt
    !pip3 install optuna

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models
from torchvision.models import *
from plotly import express as px
from collections import Counter
import numpy as np
import random

if(IN_COLAB):
    from Projekt_AIM_1.modules.dataset import IntelImageClassificationDataset
    from Projekt_AIM_1.modules.utility import NotebookPlotter, InferenceSession, Evaluator, ISO_time, apply_pruning
    from Projekt_AIM_1.modules.trainer import Trainer
   # from Projekt_AIM_1.modules.optuna_optimizer import OptunaTuner
    from Projekt_AIM_1.modules.optuna_monashara import run_optuna
else:
    from modules.dataset import IntelImageClassificationDataset
    from modules.utility import NotebookPlotter, InferenceSession, Evaluator, ISO_time, apply_pruning
    from modules.trainer import Trainer
   # from modules.optuna_optimizer import OptunaTuner
    from modules.optuna_monashara import run_optuna

torch.manual_seed(1)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True  # for reproducibility
    torch.backends.cudnn.benchmark = False

# Removed support for Tensor Units
# torch.backends.cudnn.allow_tf32 = True
# torch.backends.cuda.matmul.allow_tf32 = True

set_seed(1)

cuda


https://www.kaggle.com/datasets/puneet6060/intel-image-classification

In [ ]:
# labels, values = zip(*Counter([item[1] for item in dataset.train_dataset]).items())
# fig = px.bar(x=labels, y=values, labels={'x': 'Categories', 'y': 'Counts'}, title='Distribution of Classes')
# fig.show()

| n | label |
| --- | --- |
| 0 | buildings |
| 1 | forest |
| 2 | glacier |
| 3 | mountain |
| 4 | sea |
| 5 | street |

NotebookPlotter.plot_dataset_item_interactive(dataset.train_dataset)

In [5]:
choice = 1 # 1,2,3
freezeLayer = False
prune_model = False
USE_OPTUNA = False
OPTUNA_MO = True


if choice != 5:
    dataset = IntelImageClassificationDataset(resize=(150,150))
else:
    dataset = IntelImageClassificationDataset(resize=(384,384))

# 80% train, 20% validation for training Optuna
train_size = int(0.8 * len(dataset.train_dataset))
val_size = len(dataset.train_dataset) - train_size
train_subset, val_subset = random_split(dataset.train_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(1))

def build_model():

  # SqueezeNet 1.1
  if choice == 1:
      model = models.squeezenet1_1(weights=SqueezeNet1_1_Weights.DEFAULT)
      num_features = model.classifier[1].in_channels
      kernel_size = model.classifier[1].kernel_size
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[1] = nn.Conv2d(num_features, 6, kernel_size)


  # MobileNetV2
  elif choice == 2:
      model = models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)
      num_features = model.classifier[1].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[1] = nn.Linear(num_features, 6)

  # MobileNetV3 Small
  elif choice == 3:
      model = models.mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
      num_features = model.classifier[3].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[3] = nn.Linear(num_features, 6)

  # MobileNetV3 Large
  elif choice == 4:
      model = models.mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.DEFAULT)
      num_features = model.classifier[3].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[3] = nn.Linear(num_features, 6)

  # VisionTransformer Base 16
  elif choice == 5:
      model = models.vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_SWAG_E2E_V1)
      num_features = model.heads[0].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.heads[0] = nn.Linear(num_features, 6)

  if prune_model:
    model = apply_pruning(model, amount=0.3)

  return model

if USE_OPTUNA:
    tuner = OptunaTuner(
        model_fn=build_model,
        train_dataset=train_subset,
        val_dataset=val_subset,
        device=DEVICE
    )
    study = tuner.run(n_trials=15, seed=1)
    print("Best trial parameters:", study.best_trial.params)

    best_params = study.best_trial.params
    model = build_model()
    dataloader = DataLoader(train_subset, batch_size=best_params["batch_size"], shuffle=True)
    trainer = Trainer(model=model, lr=best_params["lr"], device=DEVICE)
    epochs = best_params["epochs"]

elif OPTUNA_MO:

    model = build_model()

    best_params, best_model_state, study = run_optuna(
        model=model,
        train_subset=train_subset,
        val_subset=val_subset,
        TrainerClass=Trainer,
        n_trials=10,
        seed=1
    )

    print("▶ Per-epoch validation accuracy (best trial):")
    best_trial = study.best_trial
    for epoch, acc in sorted(best_trial.intermediate_values.items()):
        print(f"   Epoch {epoch:2d}: {acc * 100:.2f}%")

    print(f"\n▶ Best hyperparameters: {best_params}")
    print(f"▶ Best overall accuracy: {study.best_value * 100:.2f}%")

    model.load_state_dict(best_model_state)
    dataloader = DataLoader(
        dataset.train_dataset,
        batch_size=best_params["BS_SUGGEST"],
        shuffle=True
    )
    trainer = Trainer(
        model=model,
        lr=best_params["LR_SUGGEST"],
        device=DEVICE
    )
    epochs = best_params["epochs"]



else:
    model = build_model()
    dataloader = DataLoader(dataset.train_dataset, batch_size=32, shuffle=True)
    trainer = Trainer(model=model, lr=8.841926348917726e-05, device=DEVICE)
    epochs = 25

100%|██████████| 346M/346M [00:01<00:00, 183MB/s]

Extracting files...



Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_1-b8a52dc0.pth
100%|██████████| 4.73M/4.73M [00:00<00:00, 57.5MB/s]
[I 2025-05-07 23:52:44,676] A new study created in memory with name: no-name-e3b28191-62ce-485d-9af2-2d706b010ce1


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[I 2025-05-08 01:07:41,592] Trial 9 pruned. 


▶ Per-epoch validation accuracy (best trial):
   Epoch  1: 79.69%
   Epoch  2: 83.75%
   Epoch  3: 85.57%
   Epoch  4: 87.50%
   Epoch  5: 88.39%
   Epoch  6: 88.14%
   Epoch  7: 89.06%
   Epoch  8: 89.35%
   Epoch  9: 89.31%
   Epoch 10: 89.70%
   Epoch 11: 89.95%
   Epoch 12: 89.67%
   Epoch 13: 89.63%
   Epoch 14: 90.27%
   Epoch 15: 90.63%
   Epoch 16: 90.67%
   Epoch 17: 90.45%
   Epoch 18: 90.56%
   Epoch 19: 88.92%
   Epoch 20: 90.99%

▶ Best hyperparameters: {'BS_SUGGEST': 16, 'LR_SUGGEST': 3.9292030026239486e-05}
▶ Best overall accuracy: 90.99%


KeyError: 'EPOCHS'

In [ ]:
# model.load_state_dict(torch.load(f"checkpoints/.pt"))
trainer.train(dataloader, epochs=epochs, silent=False)

In [ ]:
session = InferenceSession(model)
output = session(torch.stack(tuple(item[0] for item in dataset.test_dataset)))
Evaluator.acc(output, torch.tensor(tuple(item[1] for item in dataset.test_dataset))).item()

In [ ]:
# torch.save(model.state_dict(), f"checkpoints/{model.__class__.__name__}.pt")

## Initial Results for Model Selection

| model | accuracy | size |
| --- | --- | --- |
| ResNet18 | 0.87 | 44.7 MB |
| ResNet34 | 0.88 | 83.3 MB |
| MobileNet V2 | 0.91 | 13.6 MB |
| MobileNet V3 small | 0.90 | 9.8 MB |
| VGG19 | 0.83 | 548.1 MB |
| SqueezeNet 1.0 | 0.89 | 4.8 MB |
| DenseNet | 0.90 | 30.8 MB |
| EfficientNet B0 | 0.92 | 20.5 MB |
| ViT-b/16 | 0.73 | 330.3 MB |